# Data Quality Report Tutorial

In [1]:
import pandas as pd
import numpy as np
from dataqualityreport import dqr_table, DataQualityReport

In [2]:
df = pd.read_csv('tests/ds_salaries.csv', index_col=0)

In [3]:
# The following modifications are made ONLY TO ILLUSTRATE FUNCTIONALITY in the tutorial

# Create a partition column
df.loc[:, 'active_date'] = np.random.choice(a=range(5), size=len(df))

# Add some missing values
df.loc[df.active_date == 1, df.columns[[3,7]]] = np.nan

## Generating a DataQualityReport table

All 2-D DataFrames containing scalar values in each cell should be supported; No processing of columns is required.

In [4]:
dqr_table(df)

Building summary df...
Constructing box plots...
Spreading hist plots...
Building missing_by plots...


,Type,Unique (Cardinality = Row Count)'>Card*Unique,% MissingPartition,% MissingHeatmap,%Missing,%Zeros,%Negative,Box Plot,Robust Histogram
Column,,,,,,,,,
work_year,I,3,,,,,,,
experience_level,O,4,,,,,,,
employment_type,O,4,,,,,,,
job_title,O,48,,,,,,,
salary,I,272,,,,,,,
salary_currency,O,17,,,,,,,
salary_in_usd,I,369,,,,,,,
employee_residence,O,51,,,,,,,
remote_ratio,I,3,,,,,,,


## Understanding DQR Tables

DQR tables summarize data statistics for each column in a Pandas DataFrame. Each column is represented in a separate row. No general cross-column statistics (e.g. correlations) are computed, although there is a specific 'partition' column that has some unique treatments.

The fields in the DQR table are defined below. Many fields have tooltips with additional data available on hover.

### Type
DataType (Pandas DType) of the column

### Card(inality)
Number of unique non-NULL values in the column. Unique fields (where each non-null entry is distinct) will be denoted by a *.

### % Missing Partition
The DataFrame is partitioned by the column name provided in the `missing_by` parameter (`active_date` by default if it exists). <br> A missing % is computed for each partition and rendered as a 1-D heatmap / bar chart.

Datasets collected from production systems through ETL processes often use [horizontal](https://en.wikipedia.org/wiki/Partition_(database)) partitions as a unit of dataset for adding, updates or removal. Upon ETL failures, often times entire individual partitions can be missing, or a subset of fields will be missing for a portion of the partition.

### % Missing Heatmap 
A subset of table is randomly partitioned and a missing % is computed for each partition and rendered as a 1-D heatmap. This is useful to see correlations between missing values across columns.

### % Missing
The % of missing (null) values in the column.

### % Zeros
% of values that are zero (only valid for numeric columns)

### % Negative
% of values that are negative

### Box Plot
A [boxplot](https://en.wikipedia.org/wiki/Box_plot) is a standardized way of displaying the dataset based on the five-number summary: the minimum, the maximum, the sample median, and the first and third quartiles.

### Robust Histogram
A robust [histogram](https://en.wikipedia.org/wiki/Histogram) is an approximate representation of the distribution of numerical data, where outliers have been removed using the IQR (Inter-quantile range) method.

## Compare Two Dataframes
You can create DataQualityReports comparing N similar dataframes.  While they do not need to share all column names, `DQR_compare` will align axes for the histogram and box charts for columns that are shared in the different dataframes

In [5]:
df_train=df.sample(frac=0.8,random_state=200).iloc[:, 0:5] #random state is a seed value
df_eval=df.drop(df_train.index).iloc[:, 0:5]
print(f'df_train rows:{len(df_train)}; df_eval rows:{len(df_eval)}')

from dataqualityreport.dataqualityreport import dqr_compare
dqr_compare([df_train, df_eval], suffixes=['train', 'eval'])

df_train rows:486; df_eval rows:121
Building summary df...
Constructing box plots...
Spreading hist plots...
Building summary df...
Constructing box plots...
Spreading hist plots...


,Type,Unique (Cardinality = Row Count)'>Card*Unique,% MissingHeatmap,%Missing,%Zeros,%Negative,Box Plot,Robust Histogram
Column,,,,,,,,
employment_type-eval,O,4,,,,,,
employment_type-train,O,4,,,,,,
experience_level-eval,O,4,,,,,,
experience_level-train,O,4,,,,,,
job_title-eval,O,25,,,,,,
job_title-train,O,45,,,,,,
salary-eval,I,82,,,,,,
salary-train,I,239,,,,,,
work_year-eval,I,3,,,,,,


## DataQualityReport Rules Engine
`DataQualityReport`s use rules to define what is acceptable in terms of data quality and
how severe data quality issues are. 

An example of a rule would be:
> _Warn on missing >10% of entries for a column_

Rules provide a **severity** level:

- 0 is most severe,
- 1 moderately severe,
- ...



**Severity** can depend a lot on context - values used in linear regression must be cleaner than GBDTs, and values of the target need to be cleaner that those of features. You should consider what rules make sense for you and apply them (more on this below) - but for now, you can assume that our default rules will give you some basic useful info.

## Text Reports
The most basic output for the DataQualityReport is the text report, which is show when you try to `print` the object. It currently contains two main sections:

1) Summary counts of warnings by severity

> ``` 
S2:3, S3:10
> ```

This indicates there are 3 `S2`-level warnings, and 10 `S3`-level warnings

2) Warnings List

> ```
DataQualityWarning(level=2, field='LemasPctOfficDrugUn', msg='perc_zeros: 0.845035')
DataQualityWarning(level=2, field='NumInShelters', msg='perc_zeros: 0.544132')
DataQualityWarning(level=2, field='NumStreet', msg='perc_zeros: 0.725677')
...
> ```

The warnings list gives detail about each warning generated, including which field it pertains to, and metrics that led to the warning.

In [6]:
from dataqualityreport import DataQualityReport
dqr = DataQualityReport(df)
dqr

Building summary df...


Data Quality Report
S0:2, S2:2, S3:3
DataQualityWarning(level=0, field='employee_residence', msg='num_missing_partitions: 1, min_missing_partition: 1, max_missing_partition: 1')
DataQualityWarning(level=0, field='job_title', msg='num_missing_partitions: 1, min_missing_partition: 1, max_missing_partition: 1')
DataQualityWarning(level=2, field='company_location', msg='High percentage of a single value perc_most_freq: 0.5848434925864909, val_most_freq: US')
DataQualityWarning(level=2, field='employee_residence', msg='High percentage of a single value perc_most_freq: 0.5594989561586639, val_most_freq: US')
DataQualityWarning(level=3, field='employee_residence', msg='perc_missing: 0.2108731466227347')
DataQualityWarning(level=3, field='job_title', msg='perc_missing: 0.2108731466227347')
DataQualityWarning(level=3, field='salary', msg='num_high_10x_IQR_outliers: 21, p75: 165000, p95: 450000, max: 3.04e+07')

### Tell me your rules

In [7]:
dqr.rules

[DataQualityRule(level=0, condition='perc_missing > 0.95', fields=['perc_missing'], msg=''),
 DataQualityRule(level=0, condition='n_unique==1', fields=['n_unique', 'mean'], msg=''),
 DataQualityRule(level=0, condition='(num_missing_partitions > 0)', fields=['num_missing_partitions', 'min_missing_partition', 'max_missing_partition'], msg=''),
 DataQualityRule(level=1, condition='(perc_distinct > 0.99) & (perc_distinct < 1)', fields=['perc_distinct'], msg=''),
 DataQualityRule(level=1, condition='(perc_negative > 0) & (perc_negative < 0.05)', fields=['perc_negative', 'num_negative', 'min'], msg=''),
 DataQualityRule(level=2, condition='(perc_zeros > 0.5)', fields=['perc_zeros'], msg=''),
 DataQualityRule(level=2, condition="dtype == 'object'", fields=['dtype'], msg=''),
 DataQualityRule(level=2, condition='(perc_missing > 0.5) & (perc_missing <= 0.95)', fields=['perc_missing'], msg=''),
 DataQualityRule(level=2, condition='((n_unique > 30) | decimal_col) & (perc_most_freq > 0.4)', fields

### Only get warnings worse than S2 (e.g. S2, S1, S0)

In [8]:
print(dqr.warnings_report_str(min_dq_level=2))

Data Quality Report
S0:2, S2:2
DataQualityWarning(level=0, field='employee_residence', msg='num_missing_partitions: 1, min_missing_partition: 1, max_missing_partition: 1')
DataQualityWarning(level=0, field='job_title', msg='num_missing_partitions: 1, min_missing_partition: 1, max_missing_partition: 1')
DataQualityWarning(level=2, field='company_location', msg='High percentage of a single value perc_most_freq: 0.5848434925864909, val_most_freq: US')
DataQualityWarning(level=2, field='employee_residence', msg='High percentage of a single value perc_most_freq: 0.5594989561586639, val_most_freq: US')


### Write my own rules 
Here, we lower the threshold for perc_zeros to 10%, and get more warnings.

In [9]:
from dataqualityreport.dataqualityreport import DataQualityRule
DataQualityReport(df, rules = [
    DataQualityRule(level=2, condition='(perc_zeros > 0.1)', fields=['perc_zeros'], msg=''),
    DataQualityRule(level=2, condition="dtype == 'object'", fields=['dtype'], msg=''),
    DataQualityRule(level=2, condition='(perc_missing > 0.3) & (perc_missing <= 0.95)', 
                    fields=['perc_missing'], msg='')])

Building summary df...


Data Quality Report
S2:2
DataQualityWarning(level=2, field='active_date', msg='perc_zeros: 0.19769357495881384')
DataQualityWarning(level=2, field='remote_ratio', msg='perc_zeros: 0.20922570016474465')

### Rules reference values defined in the summary_df

In [10]:
dqr.summary_df.T

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,active_date
perc_missing,0.0,0.0,0.0,0.210873,0.0,0.0,0.0,0.210873,0.0,0.0,0.0,0.0
perc_zeros,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.209226,NaN,NaN,0.197694
num_negative,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
num_zeros,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,127.0,NaN,NaN,120.0
perc_negative,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
perc_distinct,0.004942,0.00659,0.00659,0.100209,0.448105,0.028007,0.607908,0.106472,0.004942,0.082372,0.004942,0.008237
num_low_3x_IQR_outliers,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
num_high_3x_IQR_outliers,0.0,NaN,NaN,NaN,26.0,NaN,1.0,NaN,0.0,NaN,NaN,0.0
num_low_10x_IQR_outliers,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
num_high_10x_IQR_outliers,0.0,NaN,NaN,NaN,21.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0
